<a href="https://colab.research.google.com/github/farshadabdulazeez/deep-learning-basics/blob/main/backpropagation_regression_and_classification_implementation_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Backpropagation in Regression***

# **Importing Libraries and Dataset Creation**

In [1]:
import numpy as np
import pandas as pd

# Creating a dataset with CGPA, Profile Score, and LPA (Lakhs Per Annum)
df = pd.DataFrame([[8,8,4],
                   [7,9,5],
                   [6,10,6],
                   [5,12,7]],
                  columns=['cgpa', 'profile_score', 'lpa'])


# **Parameter Initialization**

In [2]:
def initialize_parameters(layer_dims):
    """
    Initialize the parameters (weights and biases) for a neural network.

    Args:
    layer_dims -- List containing the dimensions of each layer in the network

    Returns:
    parameters -- Dictionary containing initialized weights and biases
    """
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)

    for l in range(1, L):
        parameters['W' + str(l)] = np.ones((layer_dims[l-1], layer_dims[l])) * 0.1
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))

    return parameters

# Example Usage
parameters = initialize_parameters([2, 2, 1])  # 2-input, 2-hidden, 1-output


# **Linear Forward Function**

In [3]:
def linear_forward(A_prev, W, b):
    """
    Compute the linear part of the forward propagation.

    Args:
    A_prev -- Activations from the previous layer
    W -- Weights matrix of the current layer
    b -- Bias vector of the current layer

    Returns:
    Z -- Linear combination of inputs (W.T * A_prev + b)
    """
    Z = np.dot(W.T, A_prev) + b
    return Z


# **Forward Propagation for the Full Network**

In [4]:
def L_layer_forward(X, parameters):
    """
    Forward propagation for the entire network.

    Args:
    X -- Input data
    parameters -- Dictionary containing weights and biases

    Returns:
    A -- Final output of the network
    A_prev -- Activations from the second-last layer
    """

    A = X
    L = len(parameters) // 2                  # number of layers in the neural network

    for l in range(1, L+1):
      A_prev = A
      Wl = parameters['W' + str(l)]
      bl = parameters['b' + str(l)]
      #print("A"+str(l-1)+": ", A_prev)
      #print("W"+str(l)+": ", Wl)
      #print("b"+str(l)+": ", bl)
      #print("--"*20)

      A = linear_forward(A_prev, Wl, bl)
      #print("A"+str(l)+": ", A)
      #print("**"*20)

    return A,A_prev


# **Update Parameters (Gradient Update)**

In [5]:
def update_parameters(parameters, y, y_hat, A1, X):
    """
    Update weights and biases using gradient descent.

    Args:
    parameters -- Current weights and biases
    y -- True label
    y_hat -- Predicted value
    A1 -- Activations from the hidden layer
    X -- Input features
    """
    parameters['W2'][0][0] += 0.001 * 2 * (y - y_hat) * A1[0][0]
    parameters['W2'][1][0] += 0.001 * 2 * (y - y_hat) * A1[1][0]
    parameters['b2'][0][0] += 0.001 * 2 * (y - y_hat)

    parameters['W1'][0][0] += 0.001 * 2 * (y - y_hat) * parameters['W2'][0][0] * X[0][0]
    parameters['W1'][0][1] += 0.001 * 2 * (y - y_hat) * parameters['W2'][0][0] * X[1][0]
    parameters['b1'][0][0] += 0.001 * 2 * (y - y_hat) * parameters['W2'][0][0]

    parameters['W1'][1][0] += 0.001 * 2 * (y - y_hat) * parameters['W2'][1][0] * X[0][0]
    parameters['W1'][1][1] += 0.001 * 2 * (y - y_hat) * parameters['W2'][1][0] * X[1][0]
    parameters['b1'][1][0] += 0.001 * 2 * (y - y_hat) * parameters['W2'][1][0]


# **Training the Network**

In [6]:
X = df[['cgpa', 'profile_score']].values[0].reshape(2, 1)
y = df[['lpa']].values[0][0]

# Forward Propagation
y_hat, A1 = L_layer_forward(X, parameters)
y_hat = y_hat[0][0]

# Update Parameters
update_parameters(parameters, y, y_hat, A1, X)


# **Full Training with Epochs**

In [7]:
# Initialize Parameters
parameters = initialize_parameters([2, 2, 1])
epochs = 5

for i in range(epochs):
    Loss = []

    for j in range(df.shape[0]):
        X = df[['cgpa', 'profile_score']].values[j].reshape(2, 1)
        y = df[['lpa']].values[j][0]

        # Forward Propagation
        y_hat, A1 = L_layer_forward(X, parameters)
        y_hat = y_hat[0][0]

        # Update Parameters
        update_parameters(parameters, y, y_hat, A1, X)

        # Calculate Loss
        Loss.append((y - y_hat) ** 2)

    # Print Loss per Epoch
    print('Epoch -', i+1, 'Loss -', np.array(Loss).mean())


Epoch - 1 Loss - 26.28249792398698
Epoch - 2 Loss - 19.438253848220803
Epoch - 3 Loss - 10.139874435827522
Epoch - 4 Loss - 3.385561305106485
Epoch - 5 Loss - 1.3198454128484565


In [8]:
parameters


{'W1': array([[0.273603  , 0.3993222 ],
        [0.28787155, 0.42586102]]),
 'b1': array([[0.02885522],
        [0.03133223]]),
 'W2': array([[0.42574893],
        [0.50219328]]),
 'b2': array([[0.11841278]])}

# ***Backpropagation in Classification***

In [9]:
# Import necessary libraries
import numpy as np
import pandas as pd

# Create dataset
df = pd.DataFrame([[8, 8, 1], [7, 9, 1], [6, 10, 0], [5, 5, 0]],
                  columns=['cgpa', 'profile_score', 'placed'])

In [10]:
# Function to initialize parameters for the neural network
def initialize_parameters(layer_dims):
    """
    Initialize the weights and biases for each layer of the neural network.
    Args:
        layer_dims (list): List containing dimensions of each layer in the network.
    Returns:
        parameters (dict): Dictionary containing initialized weights and biases.
    """
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)  # Number of layers in the network

    for l in range(1, L):
        parameters['W' + str(l)] = np.ones((layer_dims[l-1], layer_dims[l])) * 0.1
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))

    return parameters

In [11]:

# Function for forward propagation for a single layer
def linear_forward(A_prev, W, b):
    """
    Perform forward propagation for a single layer.
    Args:
        A_prev (numpy.ndarray): Activations from the previous layer.
        W (numpy.ndarray): Weights for the current layer.
        b (numpy.ndarray): Biases for the current layer.
    Returns:
        A (numpy.ndarray): Activation for the current layer.
    """
    Z = np.dot(W.T, A_prev) + b
    A = sigmoid(Z)
    return A

In [12]:
# Helper function for sigmoid activation
def sigmoid(Z):
    """
    Apply sigmoid activation function.
    Args:
        Z (numpy.ndarray): Linear activation input.
    Returns:
        A (numpy.ndarray): Sigmoid output.
    """
    return 1 / (1 + np.exp(-Z))

In [13]:
# Forward propagation for the entire network
def L_layer_forward(X, parameters):
    """
    Perform forward propagation through the entire network.
    Args:
        X (numpy.ndarray): Input data.
        parameters (dict): Dictionary containing weights and biases.
    Returns:
        A (numpy.ndarray): Output of the final layer.
        A_prev (numpy.ndarray): Activation from the last hidden layer.
    """
    A = X
    L = len(parameters) // 2  # Number of layers in the network

    for l in range(1, L+1):
        A_prev = A
        Wl = parameters['W' + str(l)]
        bl = parameters['b' + str(l)]
        A = linear_forward(A_prev, Wl, bl)

    return A, A_prev

In [14]:
# Function to update parameters using gradient descent
def update_parameters(parameters, y, y_hat, A1, X):
    """
    Update the weights and biases based on the error and gradients.
    Args:
        parameters (dict): Dictionary containing weights and biases.
        y (int): True label for the current training example.
        y_hat (float): Predicted output for the current training example.
        A1 (numpy.ndarray): Activation from the first hidden layer.
        X (numpy.ndarray): Input data for the current training example.
    """
    parameters['W2'][0][0] += 0.0001 * (y - y_hat) * A1[0][0]
    parameters['W2'][1][0] += 0.0001 * (y - y_hat) * A1[1][0]
    parameters['b2'][0][0] += 0.0001 * (y - y_hat)

    parameters['W1'][0][0] += 0.0001 * (y - y_hat) * parameters['W2'][0][0] * A1[0][0] * (1-A1[0][0]) * X[0][0]
    parameters['W1'][0][1] += 0.0001 * (y - y_hat) * parameters['W2'][0][0] * A1[0][0] * (1-A1[0][0]) * X[1][0]
    parameters['b1'][0][0] += 0.0001 * (y - y_hat) * parameters['W2'][0][0] * A1[0][0] * (1-A1[0][0])

    parameters['W1'][1][0] += 0.0001 * (y - y_hat) * parameters['W2'][1][0] * A1[1][0] * (1-A1[1][0]) * X[0][0]
    parameters['W1'][1][1] += 0.0001 * (y - y_hat) * parameters['W2'][1][0] * A1[1][0] * (1-A1[1][0]) * X[1][0]
    parameters['b1'][1][0] += 0.0001 * (y - y_hat) * parameters['W2'][1][0] * A1[1][0] * (1-A1[1][0])

In [16]:
# Training with epochs
parameters = initialize_parameters([2, 2, 1])  # Initialize parameters for 2-2-1 network
epochs = 50  # Number of epochs for training

for i in range(epochs):
    Loss = []

    # Iterate through each example in the dataset
    for j in range(df.shape[0]):
        X = df[['cgpa', 'profile_score']].values[j].reshape(2, 1)  # Input features
        y = df[['placed']].values[j][0]  # True label

        y_hat, A1 = L_layer_forward(X, parameters)  # Forward propagation
        y_hat = y_hat[0][0]

        update_parameters(parameters, y, y_hat, A1, X)  # Update weights and biases

        # Compute loss for the current example
        Loss.append(-y * np.log(y_hat) - (1 - y) * np.log(1 - y_hat))

    # Print the average loss for the epoch
    print('Epoch -', i+1, 'Loss -', np.array(Loss).mean())


Epoch - 1 Loss - 0.6939124309994983
Epoch - 2 Loss - 0.6939114509742006
Epoch - 3 Loss - 0.6939104713855384
Epoch - 4 Loss - 0.6939094922333021
Epoch - 5 Loss - 0.6939085135172819
Epoch - 6 Loss - 0.6939075352372682
Epoch - 7 Loss - 0.6939065573930516
Epoch - 8 Loss - 0.6939055799844227
Epoch - 9 Loss - 0.6939046030111722
Epoch - 10 Loss - 0.693903626473091
Epoch - 11 Loss - 0.6939026503699699
Epoch - 12 Loss - 0.6939016747016002
Epoch - 13 Loss - 0.6939006994677729
Epoch - 14 Loss - 0.6938997246682792
Epoch - 15 Loss - 0.6938987503029107
Epoch - 16 Loss - 0.6938977763714584
Epoch - 17 Loss - 0.6938968028737142
Epoch - 18 Loss - 0.6938958298094697
Epoch - 19 Loss - 0.6938948571785166
Epoch - 20 Loss - 0.6938938849806471
Epoch - 21 Loss - 0.6938929132156526
Epoch - 22 Loss - 0.6938919418833256
Epoch - 23 Loss - 0.693890970983458
Epoch - 24 Loss - 0.6938900005158424
Epoch - 25 Loss - 0.6938890304802711
Epoch - 26 Loss - 0.6938880608765363
Epoch - 27 Loss - 0.6938870917044309
Epoch - 28 L

In [17]:
parameters  # Final trained parameters

{'W1': array([[0.09999042, 0.09990338],
        [0.09999049, 0.0999034 ]]),
 'b1': array([[-2.63694652e-05],
        [-2.63678488e-05]]),
 'W2': array([[0.09960344],
        [0.09960349]]),
 'b2': array([[-0.00080196]])}